In [5]:
import difflib
import glob
import pickle
from pprint import pprint

from resimulate.euicc.mutation.types import MutationType
from resimulate.euicc.recorder.operation import MutationTreeNode
from resimulate.euicc.recorder.recorder import OperationRecorder

PATHS = glob.glob("recordings/*.resim")

for path in PATHS:
    with open(path, "rb") as f:
        try:
            recorder: OperationRecorder = pickle.load(f)
        except Exception as e:
            print(f"Failed to load {path}: {e}")
            continue

        print(f"Processing {path}")

    def get_successfully_mutated_data(tree: MutationTreeNode) -> bool:
        if tree.leaf:
            return tree.failure_reason is None

        has_successful_child = False

        for child in tree.children:
            is_successful = get_successfully_mutated_data(child)

            if is_successful:
                has_successful_child = True

            if (
                is_successful
                and tree.recording
                and tree.mutation_type != MutationType.NONE
            ):
                diff = difflib.ndiff(
                    tree.recording.original_apdu.data.hex(),
                    tree.recording.mutated_apdu.data.hex(),
                )
                pprint(
                    {
                        "recording": path,
                        "function": tree.func_name,
                        "mutation_type": tree.mutation_type,
                        "original_apdu": tree.recording.original_apdu.data.hex(),
                        "mutated_apdu": tree.recording.mutated_apdu.data.hex(),
                        "diff": "".join(diff),
                    }
                )

        return has_successful_child

    get_successfully_mutated_data(recorder.root)

Processing recordings/xesim-test-6_ProfileDownloadScenario_DeterministicMutationEngine.resim
{'diff': '  b  f  2  0- 0- 0',
 'function': 'get_euicc_info_1',
 'mutated_apdu': 'bf20',
 'mutation_type': <MutationType.TRUNCATE: 'truncate'>,
 'original_apdu': 'bf2000',
 'recording': 'recordings/xesim-test-6_ProfileDownloadScenario_DeterministicMutationEngine.resim'}
{'diff': '  b  f  3  8  8  2  0  3  4  3  3  0  4  0  8  0  0  8  0  0  0  0  '
         '0  0  0  0  1  9  6  0  5  c  c  a  8  1  1  0  1  e  4  5  c  a  a  '
         'e  b  4  e  7  3  d  8  a  f  0  b  8  e  2  6  c  8  c  a  7  9  d  '
         '9  f  8  3  1  0  7  2  7  3  7  0  2  e  7  4  7  2  7  5  7  0  6  '
         '8  6  f  6  e  6  5  2  e  6  3  6  f  6  d  8  4  1  0  e  a  6  1  '
         '6  5  e  b  3  2  2  5  6  3  7  d  b  4  c  c  9  3  c  5  1  5  5  '
         '1  e  e  a  e  5  f  3  7  4  0  2  b  6  5  d  6  0  2  2  a  0  a  '
         'e  8  1  1  c  8  f  6  7  0  f  5  8  f  7  3  5  8  b  a  

In [ ]:
print(
    len(
        "bf388203433040800800000000195eb0298110b2dda257b89ac26c1b169e82a473a08683107273702e74727570686f6e652e636f6d841090384cf9fb9f521ef14fc74b131752f85f3740755a1e53dbff8923890812feaace50d11a593d93d3a4e73676994d5475101ffca9c65ea7fdea01a749894c5760ec8f599f72564469ea6cdac6560f511bd56848041481370f5125d0b1d408d4c3b232e6d25e795bebfb3082028330820228a003020102021064055185218bd277dfa47d746d994324300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca381d33081d030170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302078030160603551d11040f300d880b2b0601040181ec45010101301d0603551d0e0416041418f04c54d7a617353b0662e1bd055474d19cd33f301f0603551d2304183016801481370f5125d0b1d408d4c3b232e6d25e795bebfb300a06082a8648ce3d0403020349003046022100bfbfd9279495d8a09531e55c9728c99b7b7533629fb3d564ac0c5cad09405972022100f4baf4758e94d403a6f66eb746a3c6d2fbf24e19067128182e7794fad1fa1fbaa01f801351522d472d35432d314c532d3157315a395037a108800435290611a100"
    )
)
print(
    len(
        "bf388203433040800800000000195eb0298110b2dda257b89ac26c1b169e82a473a08683107273702e74727570686f6e652e636f6d841090384cf9fb9f521ef14fc74b131752f85f3740755a1e53dbff8923890812feaace50d11a593d93d3a4e73676994d5475101ffca9c65ea7fdea01a749894c5760ec8f599f72564469ea6cdac6560f511bd56848041481370f5125d0b1d408d4c3b232e6d25e795bebfb3082028330820228a003020102021064055185218bd277dfa47d746d994324300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca381d33081d030170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302"
    )
)

1680
1260


## Get successful Mutations


In [11]:
import difflib
import glob
import pickle
from pprint import pprint

from resimulate.euicc.mutation.types import MutationType
from resimulate.euicc.recorder.operation import MutationTreeNode
from resimulate.euicc.recorder.recorder import OperationRecorder

PATHS = glob.glob("recordings/5ber*.resim")

for path in PATHS:
    with open(path, "rb") as f:
        try:
            recorder: OperationRecorder = pickle.load(f)
        except Exception as e:
            continue

        print(f"Processing {path}")

    recorder.root.print_tree()

    def get_auth_errors(tree: MutationTreeNode):
        if (
            tree.mutation_type != MutationType.NONE
            and (tree.is_successful_branch() or tree.children != [])
            and tree.func_name not in ["get_euicc_info_1", "get_euicc_challenge"]
            and tree.recording.original_apdu.data.hex()
            != tree.recording.mutated_apdu.data.hex()
        ):
            diff = difflib.ndiff(
                tree.recording.original_apdu.data.hex(),
                tree.recording.mutated_apdu.data.hex(),
            )
            pprint(
                {
                    "recording": path,
                    "function": tree.func_name,
                    "mutation_type": tree.mutation_type,
                    "original_apdu": tree.recording.original_apdu.data.hex(),
                    "mutated_apdu": tree.recording.mutated_apdu.data.hex(),
                    "response": tree.recording.response_data,
                    "diff": "".join(diff),
                }
            )

        for child in tree.children:
            get_auth_errors(child)

    get_auth_errors(recorder.root)

Processing recordings/5ber-test-2_ProfileDownloadScenario_DeterministicMutationEngine.resim
 root 
   ├── get_euicc_challenge (bitflip, ApduException)
   ├── get_euicc_challenge (zero_block, ApduException)
   ├── get_euicc_challenge (truncate, ApduException)
   ├── get_euicc_challenge (random_byte, ApduException)
   ├── get_euicc_challenge (shuffle_blocks, ApduException)
   └── get_euicc_challenge (_none, success)
      ├── get_euicc_info_1 (random_byte, ApduException)
      ├── get_euicc_info_1 (bitflip, ApduException)
      ├── get_euicc_info_1 (truncate, success)
      │  ├── authenticate_server (bitflip, ApduException)
      │  ├── authenticate_server (truncate, InvalidCertificate)
      │  │  ├── prepare_download (_none, success)
      │  │  │  └── initialiseSecureChannelRequest (shuffle_blocks, ApduException)
      │  │  ├── prepare_download (truncate, InvalidCertificate)
      │  │  └── prepare_download (bitflip, ApduException)
      │  ├── authenticate_server (_none, success)
 

In [ ]:
import difflib
import glob
import pickle
from pprint import pprint

from resimulate.euicc.mutation.types import MutationType
from resimulate.euicc.recorder.operation import MutationTreeNode
from resimulate.euicc.recorder.recorder import OperationRecorder

PATHS = glob.glob("recordings/xesim-test-7*.resim")


def print_trace(trace: list[MutationTreeNode]):
    pprint([node.func_name for node in trace])


for path in PATHS:
    with open(path, "rb") as f:
        try:
            recorder: OperationRecorder = pickle.load(f)
        except Exception as e:
            continue

        print(len(recorder.traces), "traces found in", path)

        for index, trace_set in enumerate(recorder.traces):
            for trace in trace_set:
                if (
                    trace.func_name == "authenticate_server"
                    and trace.mutation_type == MutationType.TRUNCATE
                    and trace.is_successful_branch()
                ):
                    print_trace(trace_set)
                    print(f"Found trace in {path} at index {index}")
                    break

73 traces found in recordings/xesim-test-7_ProfileDownloadScenario_DeterministicMutationEngine.resim
['get_euicc_challenge',
 'get_euicc_info_1',
 'authenticate_server',
 'prepare_download',
 'initialiseSecureChannelRequest',
 'firstSequenceOf87',
 'sequenceOf88',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86',
 'sequenceOf86']
Found trace in recordings/xesim-test-7_ProfileDownloadScenario_DeterministicMutationEngine.resim at index 14
['get_euicc_challenge', 'get_euicc_info_1', 'authenticate_server']
Found trace in recordings/xesim-test-7_ProfileDownloadScenario_DeterministicMutationEngine.resim at index 16


In [4]:
import glob
import pickle
from pprint import pprint

from resimulate.euicc.mutation.types import MutationType
from resimulate.euicc.recorder.operation import MutationTreeNode
from resimulate.euicc.recorder.recorder import OperationRecorder

PATHS = glob.glob("recordings/xesim-test-7*.resim")


def print_trace(trace: list[MutationTreeNode]):
    pprint(
        [{"function": node.func_name, "mutation": node.mutation_type} for node in trace]
    )


with open(
    "recordings/xesim-test-7_ProfileDownloadScenario_DeterministicMutationEngine.resim",
    "rb",
) as f:
    try:
        recorder: OperationRecorder = pickle.load(f)
    except Exception as e:
        exit(1)

    for i in range(7):
        traces = recorder.traces[14 - i]
        print_trace(traces)

[{'function': 'get_euicc_challenge', 'mutation': <MutationType.NONE: '_none'>},
 {'function': 'get_euicc_info_1',
  'mutation': <MutationType.TRUNCATE: 'truncate'>},
 {'function': 'authenticate_server',
  'mutation': <MutationType.TRUNCATE: 'truncate'>},
 {'function': 'prepare_download', 'mutation': <MutationType.NONE: '_none'>},
 {'function': 'initialiseSecureChannelRequest',
  'mutation': <MutationType.NONE: '_none'>},
 {'function': 'firstSequenceOf87', 'mutation': <MutationType.NONE: '_none'>},
 {'function': 'sequenceOf88', 'mutation': <MutationType.NONE: '_none'>},
 {'function': 'sequenceOf86', 'mutation': <MutationType.NONE: '_none'>},
 {'function': 'sequenceOf86', 'mutation': <MutationType.NONE: '_none'>},
 {'function': 'sequenceOf86', 'mutation': <MutationType.NONE: '_none'>},
 {'function': 'sequenceOf86', 'mutation': <MutationType.NONE: '_none'>},
 {'function': 'sequenceOf86', 'mutation': <MutationType.NONE: '_none'>},
 {'function': 'sequenceOf86', 'mutation': <MutationType.NON

In [ ]:
from resimulate.asn import asn
from pprint import pprint

response = "bf3882067ba08206773081f98008000000001959b7be83107273702e74727570686f6e652e636f6d8410c8a65776accdb4f8638143b65570a05cbf2281a6810302030182030202028303040200840d810100820400075944830237338505077f3e1f808603090200870302030088020490a92c04140b1359633d2ab469761ca9d82b2229350db722d7041481370f5125d0b1d408d4c3b232e6d25e795bebfbaa2c04140b1359633d2ab469761ca9d82b2229350db722d7041481370f5125d0b1d408d4c3b232e6d25e795bebfb8b010004030100000c0d45442d5a492d55502d30383234a01f801351522d472d35432d314c532d3157315a395037a108800435290611a1005f37407a41016bbfa6ec924a1ce51175b1647f6c1719e5b1c45ccf5af9a25562e7e0ea04dc2a45d95a746de6018bad0e89f84ea43819cee6b2d491afc948f39072d77630820239308201e0a00302010202110089086030202200000024000086963302300a06082a8648ce3d0403023079310b300906035504061302434e31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c746431153013060355040b130c45617374636f6d7065616365312930270603550403132045617374636f6d70656163652e45554d2e436f6e73756d65722e5a68756861693020170d3234303832333133353234395a180f39393939313233313233353935395a305531283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c7464312930270603550405132038393038363033303230323230303030303032343030303038363936333330333059301306072a8648ce3d020106082a8648ce3d03010703420004f3e8d74899a2416bde865bbd4b7e02d2842d2d28deaaa8820d375ccd2b36bea7779897f2bf9946966b1d58110d15a56e1aaea75872eb6b1d242cec37e8a7eba0a36b3069301f0603551d230418301680143a6b2ca4585d9c95c5947abdd3bb1b0169acef72301d0603551d0e04160414499ab8cda957ba7db854023c6500f75e3b8cb549300e0603551d0f0101ff04040302078030170603551d200101ff040d300b3009060767811201020101300a06082a8648ce3d040302034700304402203a38d9434e4110d6ccb312bf130f8c4670e2601b2cab00d9e1c5046893f07b92022033ec66a2e3af013daff426da43a5efc09b9495267338972476f0d1d0acdfc5ab308202f73082029ea00302010202105986939cf376fdd1013e8f1529307748300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3139313132313030303030305a170d3439313132303233353935395a3079310b300906035504061302434e31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c746431153013060355040b130c45617374636f6d7065616365312930270603550403132045617374636f6d70656163652e45554d2e436f6e73756d65722e5a68756861693059301306072a8648ce3d020106082a8648ce3d030107034200042fb0e6c5292e90321b0f1c69e88d5c3cf8c0e5f79e5c6f2c588552a74ab894aa0b4414b349f4616cdde9629dcaf955b7c39712eeaf46101af686d44f8b2dfc07a382013b30820137301d0603551d0e041604143a6b2ca4585d9c95c5947abdd3bb1b0169acef7230120603551d130101ff040830060101ff02010030170603551d200101ff040d300b3009060767811201020102304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302010630510603551d1e0101ff04473045a0433041a43f303d31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c74643111300f06035504051308383930383630333030160603551d11040f300d880b2b06010401838a1d010202301f0603551d2304183016801481370f5125d0b1d408d4c3b232e6d25e795bebfb300a06082a8648ce3d040302034700304402203c5fd987d7cabb5172489647a8563016be05de44b06ce285a2d6790bcdf408ca02202272886d929fc9434153068c8be3521bc371e9101a18a56e9baa7dd936592a1e"
mutated_data = asn.decode(
    "AuthenticateServerResponse",
    bytes.fromhex(response),
    check_constraints=True,
)
pprint(mutated_data)


('authenticateResponseOk',
 {'euiccCertificate': {'signature': (bytearray(b'0D\x02 :8\xd9CNA\x10\xd6'
                                               b'\xcc\xb3\x12\xbf\x13\x0f\x8cF'
                                               b'p\xe2`\x1b,\xab\x00\xd9'
                                               b'\xe1\xc5\x04h\x93\xf0{\x92'
                                               b'\x02 3\xecf\xa2\xe3\xaf'
                                               b'\x01=\xaf\xf4&\xdaC\xa5'
                                               b'\xef\xc0\x9b\x94\x95&s8'
                                               b'\x97$v\xf0\xd1\xd0\xac\xdf'
                                               b'\xc5\xab'),
                                     560),
                       'signatureAlgorithm': {'algorithm': '1.2.840.10045.4.3.2'},
                       'tbsCertificate': {'extensions': [{'critical': False,
                                                          'extnID': '2.5.29.35',
                       

In [ ]:
import difflib
import json

from IPython.display import HTML, display
from pprint import pprint

from resimulate.asn import asn

# authenticate_server_mutated = "bf3882027130408008000000001954e67b8110a42accee098234fe163aafb0837d46bf83107273702e74727570686f6e652e636f6d841071b05cf7c9c63911ea9bd771a611ca915f3740cc35a76b34ca3e5755edf9d7f667f2130c9580c96a52e8d367a4c729dba65790754547ef36c7a2e7f4186a9b8ebed3d80f9275a0b744b379d1e782bbe7ac41df041481370f5125d0b1d408d4c3b232e6d25e795bebfb308201d2308201cea003020102021064055185218bd277dfa47d746d994324300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca3817930817630170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300c0603551d0f0101ff04020302"
authenticate_server_mutated = "bf3882027130408008000000001959b7be81108f838285224dc7a4233fce0b97931a8583107273702e74727570686f6e652e636f6d8410c8a65776accdb4f8638143b65570a05c5f37402c519f80b3c6f0cec9c3c582af5974265cf09a67cd5b6ebee7205555f6129899ae02ba0ebad7aed89be4127d6e8e41e768211fc304465d23267ebca0a1a11f7a041481370f5125d0b1d408d4c3b232e6d25e795bebfb308201d2308201cea003020102021064055185218bd277dfa47d746d994324300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca3817930817630170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300c0603551d0f0101ff04020302"
authenticate_server_original = "bf3882034330408008000000001954e5df81106ab410e0d56d874bc95f478e65d6664683107273702e74727570686f6e652e636f6d8410a6bdc830e18bc5352170dc6eddb2199b5f3740fdaf7e7d13614cc7cf2f8b12c6d0cc8b7ca0fcceb2d870445cf84329964d16cb3b470e5de3f7bb984dfef55d28571cc706268e7c89e7d7d0265021a11bfbb120041481370f5125d0b1d408d4c3b232e6d25e795bebfb3082028330820228a003020102021064055185218bd277dfa47d746d994324300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca381d33081d030170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302078030160603551d11040f300d880b2b0601040181ec45010101301d0603551d0e0416041418f04c54d7a617353b0662e1bd055474d19cd33f301f0603551d2304183016801481370f5125d0b1d408d4c3b232e6d25e795bebfb300a06082a8648ce3d0403020349003046022100bfbfd9279495d8a09531e55c9728c99b7b7533629fb3d564ac0c5cad09405972022100f4baf4758e94d403a6f66eb746a3c6d2fbf24e19067128182e7794fad1fa1fbaa01f801351522d472d35432d314c532d3157315a395037a108800435290611a100"

mutated = asn.decode(
    "AuthenticateServerRequest", data=bytes.fromhex(authenticate_server_mutated)
)
original = asn.decode(
    "AuthenticateServerRequest", data=bytes.fromhex(authenticate_server_original)
)

pprint(original)
pprint(mutated)


{'ctxParams1': ('ctxParamsForCommonAuthentication',
                {'deviceInfo': {'deviceCapabilities': {}, 'tac': b'5)\x06\x11'},
                 'matchingId': 'QR-G-5C-1LS-1W1Z9P7',
                 'operationType': (b'\x80', 1)}),
 'euiccCiPKIdToBeUsed': b'\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^'
                        b'y[\xeb\xfb',
 'serverCertificate': {'signature': (bytearray(b'0F\x02!\x00\xbf\xbf\xd9'
                                               b"'\x94\x95\xd8\xa0\x951\xe5"
                                               b'\\\x97(\xc9\x9b{u3'
                                               b'b\x9f\xb3\xd5d\xac\x0c\\'
                                               b'\xad\t@Yr\x02!\x00'
                                               b'\xf4\xba\xf4u\x8e\x94\xd4\x03'
                                               b'\xa6\xf6n\xb7F\xa3\xc6\xd2'
                                               b'\xfb\xf2N\x19\x06q(\x18'
                                               b'.w\

## Trigger Truncation Bug


In [1]:
import datetime
import logging

from resimulate.asn import asn
from resimulate.euicc.card import Card
from pySim.exceptions import SwMatchError
from resimulate.euicc.exceptions import (
    ApduException,
    AuthenticateException,
    EuiccException,
    NothingToDeleteError,
)
from resimulate.euicc.models.activation_profile import ActivationProfile
from resimulate.euicc.models.reset_option import ResetOption
from resimulate.euicc.mutation.deterministic_engine import DeterministicMutationEngine
from resimulate.euicc.transport.apdu import APDUPacket
from resimulate.euicc.transport.pcsc_link import PcscLink
from resimulate.smdp.client import SmdpClient
from resimulate.euicc.mutation.types import MutationType
from resimulate.smdp.models import InitiateAuthenticationResponse

from pySim.utils import sw_match

from osmocom.utils import h2b

from pprint import pprint
import difflib


truncation_point = -327

mutation_engine = DeterministicMutationEngine()


def store_data(
    card: Card,
    request_type: str | None = None,
    response_type: str | None = None,
    request_data: dict | None = None,
    mutation_type: MutationType | None = None,
) -> dict | tuple | str | None:
    if request_data is None:
        request_data = dict()

    command_encoded = request_data
    if request_type is not None:
        print(f"Sending request: {request_type}")
        command_encoded = asn.encode(
            request_type,
            request_data,
            # check_constraints=True,
        )

    if len(command_encoded) > 65536:
        raise ValueError("Data too long")

    # mutated_data = command_encoded[:truncation_point]
    apdu = APDUPacket(cla=0x80, ins=0xE2, p1=0x91, p2=0x00, data=command_encoded)

    def handle_apdu_transmission(apdu: APDUPacket) -> tuple[str | None, str]:
        logging.debug("Sending %s", str(apdu))

        short_apdus = apdu.to_short_apdu(data_size=card.link.apdu_data_size)
        if len(short_apdus) > 1:
            logging.debug("Splitting APDU into %d short APDUs", len(short_apdus))

        for short_apdu in short_apdus:
            try:
                data, sw = card.link.send_apdu_checksw(short_apdu.to_hex())
            except SwMatchError as exception:
                return None, exception.sw_actual

        return data, sw

    if mutation_type:
        mutated_apdu = mutation_engine.mutate(apdu, mutation_type=mutation_type)
    else:
        mutated_apdu = apdu
    print(f"Mutating apdu with {mutation_type}: {mutated_apdu}")

    # diff = difflib.ndiff(apdu.data.hex(), mutated_apdu.data.hex())
    # pprint("".join(diff))

    pprint(apdu.data.hex())
    pprint(mutated_apdu.data.hex())

    # pprint(asn.decode("AuthenticateServerRequest", apdu.data, check_constraints=False))
    # pprint(
    #     asn.decode(
    #         "AuthenticateServerRequest", mutated_apdu.data, check_constraints=False
    #     )
    # )

    data, sw = handle_apdu_transmission(mutated_apdu)
    # data, sw = card.isd_r.link.send_apdu_with_mutation(
    #     caller_func_name, apdu, do_not_mutate=do_not_mutate
    # )
    print(
        f"Data send: {len(mutated_apdu.data)} bytes, received: {len(data) if data else None} bytes"
    )

    if not any([sw_match(sw, pattern) for pattern in ["9000", "61??"]]):
        raise ApduException(sw)

    if not data:
        return None

    if response_type is None:
        return data

    return asn.decode(response_type, h2b(data), check_constraints=False)


# Cert from osmo-sdmpp test
malicous_cert_spki = {
    "subjectPublicKeyInfo": bytearray(
        b"0Y0\x13\x06\x07*\x86"
        b"H\xce=\x02\x01\x06\x08*"
        b"\x86H\xce="
        b"\x03\x01\x07\x03"
        b"B\x00\x04M\xfe\xd4\xf4i"
        b"G\x91\xbf\x16"
        b"\x95\xce\xa00z5\xb4\x18"
        b"\x01\x96\x958{\xb7[}"
        b"$G\xb6\xb5 \x9f\x04E"
        b"\xaeN^R\x1c\xd18\x88"
        b'\xd7_\xe0|\x85\x80"*'
        b"\xe2\r\xba\xac"
        b"\x1dw\xcdv0I\x93B"
        b"\x1b\xd79"
    ),
}

valid_cert = {
    "signature": bytearray(
        b"\x03I\x000F\x02!\x00\xbf\xbf\xd9'\x94\x95\xd8\xa0"
        b"\x951\xe5\\\x97(\xc9\x9b{u3b\x9f\xb3\xd5d"
        b"\xac\x0c\\\xad\t@Yr\x02!\x00\xf4\xba\xf4u\x8e"
        b"\x94\xd4\x03\xa6\xf6n\xb7F\xa3\xc6\xd2\xfb"
        b"\xf2N\x19\x06q(\x18.w\x94\xfa\xd1\xfa\x1f\xba"
    ),
    "signatureAlgorithm": bytearray(b"0\n\x06\x08*\x86H\xce=\x04\x03\x02"),
    "tbsCertificate": {
        "extensions": bytearray(
            b"0\x81\xd00\x17\x06\x03U"
            b"\x1d \x01\x01\xff\x04\r0"
            b"\x0b0\t\x06\x07g\x81\x12"
            b"\x01\x02\x01\x040M\x06\x03"
            b"U\x1d\x1f\x04F0D0B\xa0@\xa0"
            b">\x86<http://gsma-crl.symauth.c"
            b"om/offlineca/gsma-rsp2-root-ci1."
            b"crl0\x0e\x06\x03U\x1d\x0f\x01\x01"
            b"\xff\x04\x04\x03\x02\x07\x800"
            b"\x16\x06\x03U\x1d\x11\x04\x0f"
            b"0\r\x88\x0b+\x06\x01\x04"
            b"\x01\x81\xecE\x01\x01\x010"
            b"\x1d\x06\x03U\x1d\x0e\x04\x16"
            b"\x04\x14\x18\xf0LT\xd7\xa6"
            b"\x175;\x06b\xe1\xbd\x05Tt\xd1\x9c"
            b"\xd3?0\x1f\x06\x03U\x1d#\x04\x180"
            b"\x16\x80\x14\x817\x0fQ%"
            b"\xd0\xb1\xd4\x08\xd4\xc3\xb22"
            b"\xe6\xd2^y[\xeb\xfb"
        ),
        "issuer": bytearray(
            b"0D1\x180\x16\x06\x03U\x04\n\x13"
            b"\x0fGSM Association1(0&\x06\x03U\x04"
            b"\x03\x13\x1fGSM Association - RSP2 Ro"
            b"ot CI1"
        ),
        "serialNumber": bytearray(b"\x02\x10d\x05Q\x85!\x8b\xd2w\xdf\xa4}tm\x99C$"),
        "signature": bytearray(b"0\n\x06\x08*\x86H\xce=\x04\x03\x02"),
        "subject": bytearray(
            b"0l1\x0b0\t\x06\x03U\x04\x06\x13"
            b"\x02GB1\x0f0\r\x06\x03U\x04\x07"
            b"\x13\x06London1%0#\x06\x03U\x04"
            b"\n\x13\x1cTP Global Operations Limi"
            b"ted1\x0c0\n\x06\x03U\x04\x0b"
            b"\x13\x03RSP1\x170\x15\x06\x03U"
            b"\x04\x03\x13\x0e1GLOBAL SM-DP+"
        ),
        "subjectPublicKeyInfo": bytearray(
            b"0Y0\x13\x06\x07*\x86"
            b"H\xce=\x02\x01\x06\x08*"
            b"\x86H\xce="
            b"\x03\x01\x07\x03"
            b"B\x00\x04o\xbfHY!"
            b"XI\x0c7\xd6S\x80\xf6"
            b"\xcf\x1c\x15\xec"
            b"\x02\xfb\x0e\x04"
            b"\xcb\x97\x88\xa3"
            b"qs\x1b\xd2\xaf\xa6'\xea"
            b"\x90l\x12\xbd"
            b"\x02\xb7J\x05"
            b"\xc0\x81c\xce"
            b"\x1e\xff\xdc5s2\xdc\xd9"
            b'\xc3\x13\x97)\x92"!\xd6'
            b"\xa7\xa5\x1c"
        ),
        "validity": bytearray(b"0\x1e\x17\r240227000000Z\x17\r270226235959Z"),
        "version": bytearray(b"\xa0\x03\x02\x01\x02"),
    },
}

server_signed_1 = None


def authenticate_server(
    card: Card,
    authentication: InitiateAuthenticationResponse,
    matching_id: str,
    imei: str | None = None,
    mutation_type: MutationType | None = None,
) -> dict:
    device_info = {"tac": bytes.fromhex("35290611"), "deviceCapabilities": dict()}
    if imei:
        device_info["imei"] = imei

    # malicious_cert = asn.decode(
    #     "Certificate", authentication.server_certificate, check_constraints=False
    # )

    # valid_cert["subjectPublicKeyInfo"] = malicious_cert["tbsCertificate"][
    #     "subjectPublicKeyInfo"
    # ]

    # malicious_cert["tbsCertificate"]["subjectPublicKeyInfo"] = malicous_cert_spki[
    #     "subjectPublicKeyInfo"
    # ]

    # encoded_cert = asn.encode("Certificate", valid_cert, check_constraints=False)

    # decoded_ss1 = asn.decode(
    #     "ServerSigned1",
    #     authentication.server_signed_1,
    #     check_constraints=False,
    # )
    # pprint(decoded_ss1)

    # global server_signed_1

    # if server_signed_1 is not None:
    #     decoded_ss1["serverAddress"] = server_signed_1["serverAddress"]
    # else:
    #     server_signed_1 = decoded_ss1

    # encoded_ss1 = asn.encode(
    #     "ServerSigned1",
    #     decoded_ss1,
    #     check_constraints=False,
    # )

    request = {
        "serverSigned1": authentication.server_signed_1,
        "serverSignature1": authentication.server_signature_1,
        "euiccCiPKIdToBeUsed": authentication.euicc_ci_pki_to_be_used,
        "serverCertificate": authentication.server_certificate,
        "ctxParams1": (
            "ctxParamsForCommonAuthentication",
            {
                "matchingId": matching_id,
                "deviceInfo": device_info,
            },
        ),
    }

    print(f"AuthenticateServerRequest: {request}")

    data = store_data(
        card=card,
        request_type="AuthenticateServerRequest",
        request_data=request,
        mutation_type=mutation_type,
    )

    response = asn.decode(
        "AuthenticateServerResponse",
        bytes.fromhex(data),
        check_constraints=False,
    )

    print(f"AuthenticateServerResponse: {response}")

    result, error = response
    if result == "authenticateResponseError":
        code = error.get("authenticateErrorCode")
        raise AuthenticateException.raise_from_code(code)

    return data


def clear_card(card: Card):
    notifications = card.isd_r.retrieve_notification_list()
    if notifications:
        card.isd_r.process_notifications(notifications)

    try:
        card.isd_r.reset_euicc_memory(reset_options=ResetOption.all())
    except NothingToDeleteError:
        pass


with PcscLink(apdu_data_size=255) as link:
    card = Card(link)

    profile_1 = ActivationProfile.from_activation_code(
        "LPA:1$rsp.truphone.com$QRF-SPEEDTEST"  # "LPA:1$rsp-eu.redteamobile.com$5901981126831169"  # "LPA:1$localhost:8000$TS48v5_SAIP2.1B_NoBERTLV"
    )

    smdp_address = profile_1.smdp_address
    if not smdp_address:
        smdp_address = card.isd_r.get_configured_data().default_dp_address

    assert smdp_address, "No SMDP address found in profile or card configuration."

    smdp_client = SmdpClient(
        smdp_address=smdp_address, verify_ssl=False, protocol="https"
    )

    clear_card(card)
    euicc_challenge = card.isd_r.get_euicc_challenge()
    euicc_info_1 = card.isd_r.get_euicc_info_1()
    init_auth = smdp_client.initiate_authentication(euicc_challenge, euicc_info_1)
    try:
        authenticate_server(
            card,
            init_auth,
            profile_1.matching_id,
            mutation_type=MutationType.BITFLIP,
        )
    except Exception as e:
        logging.error(f"Authentication failed: {e}")

    clear_card(card)

    profile = ActivationProfile.from_activation_code(
        "LPA:1$rsp.truphone.com$QRF-SPEEDTEST"  # "LPA:1$localhost:8000$TS48v5_SAIP2.1B_NoBERTLV"
    )

    smdp_address = profile.smdp_address
    if not smdp_address:
        smdp_address = card.isd_r.get_configured_data().default_dp_address

    assert smdp_address, "No SMDP address found in profile or card configuration."

    smdp_client = SmdpClient(
        smdp_address=smdp_address, verify_ssl=False, protocol="https"
    )

    logging.debug(f"Downloading profile from {smdp_address}")
    euicc_challenge = card.isd_r.get_euicc_challenge()

    euicc_info_1 = card.isd_r.get_euicc_info_1()

    init_auth = smdp_client.initiate_authentication(euicc_challenge, euicc_info_1)
    transaction_id = init_auth.transaction_id

    auth_server = authenticate_server(
        card, init_auth, profile.matching_id, mutation_type=MutationType.TRUNCATE
    )
    # auth_server = card.isd_r.authenticate_server(init_auth, profile.matching_id)
    authenticate_client = smdp_client.authenticate_client(transaction_id, auth_server)
    prepare_download = card.isd_r.prepare_download(
        authenticate_client, profile.confirmation_code
    )

    get_bpp_response = smdp_client.get_bound_profile_package(
        transaction_id,
        prepare_download,
    )
    notification = card.isd_r.load_bound_profile_package(get_bpp_response)
    print(notification)
    print("Profile download completed successfully.")
    print("SUCCESS!!!!!!!")

Connecting to SMDP+ server at rsp.truphone.com
SSL verification is disabled


ERROR:root:Authentication failed: 6a88 -> Referenced data not found


AuthenticateServerRequest: {'serverSigned1': b'0@\x80\x08\x00\x00\x00\x00\x19m\xb1\xce\x81\x10\xc9D\xb8\xca\x8b6\x83\xdd\x1f\x18\x00f\xd8+\x97\t\x83\x10rsp.truphone.com\x84\x10\xad\xcb1\xe5\xfe\xbb\xaee\xce#\x92-Z\x98\xed\xbb', 'serverSignature1': b'_7@uv\xa8\xbd\xe5\x90\x1dl\xc4\xb4(0\x8f\x91\xb3Nv\xbc\xa2\x89$88\xcf\xcaj\xbaa\xdb{\xa4\x15\ryy\xae3\xeby\xf4\x11\xea]\x0cDq\xe4\xf41\xfb\xa0\xf0Y|[Z\xdf\xc7\x8em\xa9\xba\x0e!', 'euiccCiPKIdToBeUsed': b'\x04\x14\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^y[\xeb\xfb', 'serverCertificate': b'0\x82\x02\x830\x82\x02(\xa0\x03\x02\x01\x02\x02\x10d\x05Q\x85!\x8b\xd2w\xdf\xa4}tm\x99C$0\n\x06\x08*\x86H\xce=\x04\x03\x020D1\x180\x16\x06\x03U\x04\n\x13\x0fGSM Association1(0&\x06\x03U\x04\x03\x13\x1fGSM Association - RSP2 Root CI10\x1e\x17\r240227000000Z\x17\r270226235959Z0l1\x0b0\t\x06\x03U\x04\x06\x13\x02GB1\x0f0\r\x06\x03U\x04\x07\x13\x06London1%0#\x06\x03U\x04\n\x13\x1cTP Global Operations Limited1\x0c0\n\x06\x03U\x04\x0b\x13\x03RSP1\x170\x15\

In [ ]:
with PcscLink(apdu_data_size=255) as link:
    card = Card(link)

Connecting to SMDP+ server at rsp.truphone.com
SSL verification is disabled
AuthenticateServerRequest: {'serverSigned1': b'0@\x80\x08\x00\x00\x00\x00\x19j\xe4\xa4\x81\x10\xef\xeb\xbc\xbe(W\x05\x8b\xe3\x9c\xe7vtI\xe8\\\x83\x10rsp.truphone.com\x84\x103\xbc\xdf3,T\xa9\x17\xd8C\x1c\xf5\x1aN\x99\x00', 'serverSignature1': b'_7@\x8b\xc4\xc0\xed\x1d\x08g\xe6=J\x15N\x8d\xd9\xc4\xe2F\x15\xd7:\xa2\xff\x110\xc2V\x9c\xec\xfb\x8b\xa8\x8b\xf3Sx\x85%\x932\x19s\xeb\xb0\xbfew\xeb*\xe9\xaa\xffNc;\x0erGf\xf7\xc6\xd6%a\x9e', 'euiccCiPKIdToBeUsed': b'\x04\x14\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^y[\xeb\xfb', 'serverCertificate': b'0\x82\x02\x830\x82\x02(\xa0\x03\x02\x01\x02\x02\x10d\x05Q\x85!\x8b\xd2w\xdf\xa4}tm\x99C$0\n\x06\x08*\x86H\xce=\x04\x03\x020D1\x180\x16\x06\x03U\x04\n\x13\x0fGSM Association1(0&\x06\x03U\x04\x03\x13\x1fGSM Association - RSP2 Root CI10\x1e\x17\r240227000000Z\x17\r270226235959Z0l1\x0b0\t\x06\x03U\x04\x06\x13\x02GB1\x0f0\r\x06\x03U\x04\x07\x13\x06London1%0#\x06\x03U\x04\n\x

In [ ]:
import difflib
import glob
import pickle
from pprint import pprint
from resimulate.asn import asn

from resimulate.euicc.mutation.types import MutationType
from resimulate.euicc.recorder.operation import MutationTreeNode
from resimulate.euicc.recorder.recorder import OperationRecorder

PATHS = glob.glob("recordings/*.resim")

errors = {}

for path in PATHS:
    with open(path, "rb") as f:
        try:
            recorder: OperationRecorder = pickle.load(f)
        except Exception as e:
            continue

        def get_errors(tree: MutationTreeNode):
            if tree.failure_reason:
                errors.setdefault(tree.failure_reason, []).append(
                    {
                        "function": tree.func_name,
                        "mutation_type": tree.mutation_type,
                        "node": tree,
                    }
                )
            for child in tree.children:
                get_errors(child)

        get_errors(recorder.root)

# Count occurrences and mutation types for each error
error_summary = {}
for error, occurrences in errors.items():
    mutation_types = [occ["mutation_type"] for occ in occurrences]
    error_summary[error] = {
        "count": len(occurrences),
        "mutation_types": sorted(set(mutation_types)),
        "functions": sorted(set(occ["function"] for occ in occurrences)),
    }
# Sort errors by count
sorted_errors = sorted(error_summary.items(), key=lambda x: x[1]["count"], reverse=True)
pprint(sorted_errors)
response_data = errors["UnsupportedCurve"][24]["node"].recording.mutated_apdu.data.hex()
pprint(response_data)


[('ApduException',
  {'count': 857,
   'functions': ['authenticate_server',
                 'firstSequenceOf87',
                 'get_euicc_challenge',
                 'get_euicc_info_1',
                 'initialiseSecureChannelRequest',
                 'prepare_download',
                 'sequenceOf86',
                 'sequenceOf88'],
   'mutation_types': [<MutationType.NONE: '_none'>,
                      <MutationType.BITFLIP: 'bitflip'>,
                      <MutationType.RANDOM_BYTE: 'random_byte'>,
                      <MutationType.SHUFFLE_BLOCKS: 'shuffle_blocks'>,
                      <MutationType.TRUNCATE: 'truncate'>,
                      <MutationType.ZERO_BLOCK: 'zero_block'>]}),
 ('SCP03TSecurityError',
  {'count': 380,
   'functions': ['firstSequenceOf87', 'sequenceOf86', 'sequenceOf88'],
   'mutation_types': [<MutationType.NONE: '_none'>,
                      <MutationType.BITFLIP: 'bitflip'>,
                      <MutationType.RANDOM_BYTE: 'random_byte'

In [ ]:
from pprint import pprint
from resimulate.asn import asn

original = "bf3882033d3040800800000000196daea3811001cc896be3792207cf047f78da25346883107273702e74727570686f6e652e636f6d8410d96ba688be271cf46368b646170eec755f3740d995261332622cbb0ed514cbfef3fe5b48ce8655e3aac30b223dbc89d5854a37b54ecabeb492ceddf7726f474b3041d3ccb3aef3bcccdb5b6e0350983990f7f4041481370f5125d0b1d408d4c3b232e6d25e795bebfb3082028330820228a003020102021064055185218bd277dfa47d746d994324300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca381d33081d030170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302078030160603551d11040f300d880b2b0601040181ec45010101301d0603551d0e0416041418f04c54d7a617353b0662e1bd055474d19cd33f301f0603551d2304183016801481370f5125d0b1d408d4c3b232e6d25e795bebfb300a06082a8648ce3d0403020349003046022100bfbfd9279495d8a09531e55c9728c99b7b7533629fb3d564ac0c5cad09405972022100f4baf4758e94d403a6f66eb746a3c6d2fbf24e19067128182e7794fad1fa1fbaa019800d5152462d535045454454455354a108800435290611a100"
pprint(
    asn.decode(
        "AuthenticateServerRequestOriginal",
        bytes.fromhex(original),
        check_constraints=False,
    )
)

manipulated = "bf3882033d3040800800000000196daea3811001cc896be3792207cf047f785a25346883107273702e74727570686f6e652e636f6d8410d96ba688be271cb46368b646170eec755f3740d995261332622cbb0ed514cbfef3fe5b48ce8675e3aac30b223dbc89d5854a37b54ecabeb492ceddf7726f474b3041d3ccb3bef3bcccdb5b6e0350983990f7f4041481370f5125d0b1d408d4c3b232e6d256795bebfb3082028330820228a003020102021064055185218bd277dfa47d706d994324300a06082a8648ce3d040302304431183016060355040a130f47514d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca381d33081d030170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302078030160603551d11040f300d880b2b0601040181ec45010101301d0603551d0e0416041418f04c54d7a617353b0662e1bd055474d19cd33f301f0603551d2304183016801481370f5125d0b1d408d4c3b232e6d25e795bebfb300a06082a8648ce3d0403020349003046022100bfbfd9279495d8a09531e55c9728c99b7b7533629fb3d564ac0c5cad09405972022100f4baf4758e94d403a6f66eb746a3c6d2fbf24e19067128182e7794fad1fa1fbaa019800d5152462d535045454454455354a108800435290611a100"
pprint(
    asn.decode(
        "AuthenticateServerRequestOriginal",
        bytes.fromhex(manipulated),
        check_constraints=False,
    )
)

{'ctxParams1': ('ctxParamsForCommonAuthentication',
                {'deviceInfo': {'deviceCapabilities': {}, 'tac': b'5)\x06\x11'},
                 'matchingId': 'QRF-SPEEDTEST',
                 'operationType': (b'\x80', 1)}),
 'euiccCiPKIdToBeUsed': bytearray(b'\x04\x14\x817\x0fQ%\xd0\xb1\xd4\x08\xd4'
                                  b'\xc3\xb22\xe6\xd2^y[\xeb\xfb'),
 'serverCertificate': {'signature': bytearray(b'\x03I\x000F\x02!\x00'
                                              b"\xbf\xbf\xd9'\x94\x95\xd8\xa0"
                                              b'\x951\xe5\\\x97(\xc9\x9b{u3b'
                                              b'\x9f\xb3\xd5d\xac\x0c\\\xad'
                                              b'\t@Yr\x02!\x00\xf4\xba\xf4u\x8e'
                                              b'\x94\xd4\x03\xa6\xf6n\xb7F'
                                              b'\xa3\xc6\xd2\xfb\xf2N\x19\x06'
                                              b'q(\x18.w\x94\xfa\xd1'
              

In [36]:
mutated_data = "bf38820686a08206823081f9800800000000195ff51583107273702e74727570686f6e652e636f6d841067c43728d26f7544ff6baf0d02d36ef4bf2281a6810302030182030202028303040200840d81010082040006aa0e830209e68505077f3e1f808603090200870302030088020490a92c0414c1d4af8957f48cd51a47fef73123a04011d3d69e041481370f5125d0b1d408d4c3b232e6d25e795bebfbaa2c0414c1d4af8957f48cd51a47fef73123a04011d3d69e041481370f5125d0b1d408d4c3b232e6d25e795bebfb8b010004030100000c0d45442d5a492d55502d30383236a01f801351522d472d35432d314c532d3157315a395037a108800435290611a10a000000000000000000005f3740a4340b83b9370c24742c6677aebdd0f3e973e5b92947c147c5e17d80e2fabd450dfc8388e399eb3aac831becfb706833d8367ab95a44681f5d9f646ff5ce92bd3082023a308201e0a00302010202110089086030202200000025000008642402300a06082a8648ce3d0403023079310b300906035504061302434e31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c746431153013060355040b130c45617374636f6d7065616365312930270603550403132045617374636f6d70656163652e45554d2e436f6e73756d65722e5a68756861693020170d3235303331383036303832315a180f39393939313233313233353935395a305531283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c7464312930270603550405132038393038363033303230323230303030303032353030303030383634323434363059301306072a8648ce3d020106082a8648ce3d03010703420004234c0403eb9a99089e9352b92886268bc03e14b2869a2d134a6b26ed525dd677d6cf18c02f8d91e8bafd16ffd4a71928ecb8812b9ab087ea94ce55434547f471a36b3069301f0603551d230418301680143a6b2ca4585d9c95c5947abdd3bb1b0169acef72301d0603551d0e04160414dd5a66f806447980246d7edc5949e966f7cd2244300e0603551d0f0101ff04040302078030170603551d200101ff040d300b3009060767811201020101300a06082a8648ce3d04030203480030450221009dbf7924f555dc3ce460ca472dccf087ef4aa6077d41d122c06fa7fc27004148022046c13be14bc1742abd602230dbf27490a3e1a5115de5b544e1be897ad39d7430308202f73082029ea00302010202105986939cf376fdd1013e8f1529307748300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3139313132313030303030305a170d3439313132303233353935395a3079310b300906035504061302434e31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c746431153013060355040b130c45617374636f6d7065616365312930270603550403132045617374636f6d70656163652e45554d2e436f6e73756d65722e5a68756861693059301306072a8648ce3d020106082a8648ce3d030107034200042fb0e6c5292e90321b0f1c69e88d5c3cf8c0e5f79e5c6f2c588552a74ab894aa0b4414b349f4616cdde9629dcaf955b7c39712eeaf46101af686d44f8b2dfc07a382013b30820137301d0603551d0e041604143a6b2ca4585d9c95c5947abdd3bb1b0169acef7230120603551d130101ff040830060101ff02010030170603551d200101ff040d300b3009060767811201020102304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302010630510603551d1e0101ff04473045a0433041a43f303d31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c74643111300f06035504051308383930383630333030160603551d11040f300d880b2b06010401838a1d010202301f0603551d2304183016801481370f5125d0b1d408d4c3b232e6d25e795bebfb300a06082a8648ce3d040302034700304402203c5fd987d7cabb5172489647a8563016be05de44b06ce285a2d6790bcdf408ca02202272886d929fc9434153068c8be3521bc371e9101a18a56e9baa7dd936592a1e"
mutated_decode = asn.decode(
    "AuthenticateServerResponse",
    bytes.fromhex(mutated_data),
    check_constraints=True,
)

pprint("Mutated")
pprint(mutated_decode[1]["euiccSigned1"]["serverChallenge"].hex())

original_data = "bf3882067ca08206783081f980080000000019606ca083107273702e74727570686f6e652e636f6d8410a8b388531acc81cc9dd030377fc1e348bf2281a6810302030182030202028303040200840d81010082040006aa0e830209e68505077f3e1f808603090200870302030088020490a92c0414c1d4af8957f48cd51a47fef73123a04011d3d69e041481370f5125d0b1d408d4c3b232e6d25e795bebfbaa2c0414c1d4af8957f48cd51a47fef73123a04011d3d69e041481370f5125d0b1d408d4c3b232e6d25e795bebfb8b010004030100000c0d45442d5a492d55502d30383236a01f801351522d472d35432d314c532d3157315a395037a108800435290611a1005f3740ca9ddabea67483581a27e82cb4c7b1539cf4e52da50ac305b5ecfdbec5d5ef5e3e24cc27d4b5b3fb55bf6f3a111a3a7eb044c15ecb1d3e7c0bb8d26f67e699383082023a308201e0a00302010202110089086030202200000025000008642402300a06082a8648ce3d0403023079310b300906035504061302434e31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c746431153013060355040b130c45617374636f6d7065616365312930270603550403132045617374636f6d70656163652e45554d2e436f6e73756d65722e5a68756861693020170d3235303331383036303832315a180f39393939313233313233353935395a305531283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c7464312930270603550405132038393038363033303230323230303030303032353030303030383634323434363059301306072a8648ce3d020106082a8648ce3d03010703420004234c0403eb9a99089e9352b92886268bc03e14b2869a2d134a6b26ed525dd677d6cf18c02f8d91e8bafd16ffd4a71928ecb8812b9ab087ea94ce55434547f471a36b3069301f0603551d230418301680143a6b2ca4585d9c95c5947abdd3bb1b0169acef72301d0603551d0e04160414dd5a66f806447980246d7edc5949e966f7cd2244300e0603551d0f0101ff04040302078030170603551d200101ff040d300b3009060767811201020101300a06082a8648ce3d04030203480030450221009dbf7924f555dc3ce460ca472dccf087ef4aa6077d41d122c06fa7fc27004148022046c13be14bc1742abd602230dbf27490a3e1a5115de5b544e1be897ad39d7430308202f73082029ea00302010202105986939cf376fdd1013e8f1529307748300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3139313132313030303030305a170d3439313132303233353935395a3079310b300906035504061302434e31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c746431153013060355040b130c45617374636f6d7065616365312930270603550403132045617374636f6d70656163652e45554d2e436f6e73756d65722e5a68756861693059301306072a8648ce3d020106082a8648ce3d030107034200042fb0e6c5292e90321b0f1c69e88d5c3cf8c0e5f79e5c6f2c588552a74ab894aa0b4414b349f4616cdde9629dcaf955b7c39712eeaf46101af686d44f8b2dfc07a382013b30820137301d0603551d0e041604143a6b2ca4585d9c95c5947abdd3bb1b0169acef7230120603551d130101ff040830060101ff02010030170603551d200101ff040d300b3009060767811201020102304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302010630510603551d1e0101ff04473045a0433041a43f303d31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c74643111300f06035504051308383930383630333030160603551d11040f300d880b2b06010401838a1d010202301f0603551d2304183016801481370f5125d0b1d408d4c3b232e6d25e795bebfb300a06082a8648ce3d040302034700304402203c5fd987d7cabb5172489647a8563016be05de44b06ce285a2d6790bcdf408ca02202272886d929fc9434153068c8be3521bc371e9101a18a56e9baa7dd936592a1e"
pprint("Original")
original = asn.decode(
    "AuthenticateServerResponse",
    bytes.fromhex(original_data),
    check_constraints=True,
)
pprint(original[1]["euiccSigned1"]["serverChallenge"].hex())

'Mutated'
'67c43728d26f7544ff6baf0d02d36ef4'
'Original'
'a8b388531acc81cc9dd030377fc1e348'
